In [1]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [2]:
!pip install -q torch==2.4.1+cu121 torchvision==0.19.1+cu121 torchaudio==2.4.1+cu121 --extra-index-url https://download.pytorch.org/whl/cu121

In [4]:
!pip install  -U -q git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git datasets qwen-vl-utils wandb accelerate

In [ ]:
import os
import wandb
from huggingface_hub import login

login(token=os.getenv("HUGGINGFACE_TOKEN"))

wandb.login(key=os.getenv("WANDB_API_KEY"))


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tungpthe187331 (tungpthe187331-fpt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
import torch
import wandb
from PIL import Image
from datasets import load_dataset
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor
from qwen_vl_utils import process_vision_info
from trl import SFTConfig
from trl import SFTTrainer

In [7]:
train_dataset, eval_dataset = load_dataset("phmtung/Qwen2-VL-OCR-ViDocs", split=["train", "eval"])

README.md:   0%|          | 0.00/500 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

eval-00000-of-00001.parquet:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5100 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/45 [00:00<?, ? examples/s]

In [8]:
def format_data(sample):
    images = sample["image"] if isinstance(sample["image"], list) else [sample["image"]]
    user_content = [{"type": "image", "image": img} for img in images]
    user_content.append({"type": "text", "text": sample["query"]})

    return [
        {
            "role": "user",
            "content": user_content,
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": sample["label"]}],
        },
    ]

In [9]:
train_dataset = [format_data(sample) for sample in train_dataset]
eval_dataset = [format_data(sample) for sample in eval_dataset]

In [10]:
eval_dataset[0]

[{'role': 'user',
  'content': [{'type': 'image',
    'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2829x1910>},
   {'type': 'text',
    'text': 'You are a highly precise assistant specializing in Vietnamese OCR. Your task is to **accurately extract text** from vehicle inspection report image. Please follow these strict requirements:  \n\n- Extract text **exactly as it appears** in the image, including capitalization, special characters, and formatting.  \n- **Do not modify, infer, or guess** any missing values. If a field is unreadable or missing, return an empty string ("").  \n- Ensure all extracted text is **error-free** and corresponds correctly to the predefined fields.\n- Maintain the exact JSON structure and **output only valid JSON**.\n- **Note:** The image may be rotated, distorted, damaged, low-resolution, or affected by lighting inconsistencies (e.g., glare, shadows, overexposure).\n\n### **JSON Output Format:**\n{\n    "PHƯƠNG TIỆN (VEHICLE)": {\n        

In [11]:
eval_dataset[34]

[{'role': 'user',
  'content': [{'type': 'image',
    'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=800x600>},
   {'type': 'image',
    'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2048x1221>},
   {'type': 'text',
    'text': 'You are a highly precise assistant specializing in Vietnamese OCR. Your task is to **accurately extract text** from two images of a Citizen Identity Card: the front side and the back side. Please follow these strict requirements:  \n\n- Extract text **exactly as it appears** in the image, including capitalization, special characters, and formatting.  \n- **Do not modify, infer, or guess** any missing values. If a field is unreadable or missing, return an empty string ("").  \n- Ensure all extracted text is **error-free** and corresponds correctly to the predefined fields.\n- Maintain the exact JSON structure and **output only valid JSON**.\n- **Note:** The image may be rotated, distorted, damaged, low-resolution, or affected b

In [12]:
model_id = "Qwen/Qwen2-VL-2B-Instruct"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="cuda:0",
    torch_dtype=torch.bfloat16
)

min_pixels = 256 * 28 * 28
max_pixels = 1280 * 28 * 28

processor = Qwen2VLProcessor.from_pretrained(model_id, min_pixels=min_pixels, max_pixels=max_pixels)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [13]:
!nvidia-smi

Fri May 16 04:37:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A800 80GB PCIe          On  |   00000000:01:00.0 Off |                    0 |
| N/A   50C    P0             75W /  300W |    5139MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [14]:
# Configure training arguments
training_args = SFTConfig(
    output_dir="./Qwen2",  
    num_train_epochs=1,  
    per_device_train_batch_size=2,  
    per_device_eval_batch_size=2,  
    gradient_accumulation_steps=8,  
    gradient_checkpointing=True,  
    # Optimizer and scheduler settings
    optim="adamw_torch_fused",  
    learning_rate=1e-5, 
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr": 1e-6},
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
    # Logging and evaluation
    logging_steps=30,  
    eval_steps=30,  
    eval_strategy="steps", 
    save_strategy="steps",  
    save_steps=60, 
    metric_for_best_model="eval_loss",  
    greater_is_better=False,  
    load_best_model_at_end=True, 
    # Mixed precision and gradient settings
    bf16=True,  
    tf32=True,
    weight_decay=0.05,
    max_grad_norm=1.0, 
    warmup_ratio=0.03,  
    # Hub and reporting
    push_to_hub=True, 
    report_to="wandb",  
    # Gradient checkpointing settings
    gradient_checkpointing_kwargs={"use_reentrant": False},  
    # Dataset configuration
    dataset_text_field="", 
    dataset_kwargs={"skip_prepare_dataset": True},  
    max_seq_length=None # No truncation applied
)

training_args.remove_unused_columns = False  

In [ ]:
import wandb

wandb.init(
    project="Qwen2-VL-ViDocs",  
    name="Qwen2-VL-ViDocs",  
    config=training_args,
)

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [17]:
# Create a data collator to encode text and image pairs
def collate_fn(examples):
    # Get the texts and images, and apply the chat template
    texts = [
        processor.apply_chat_template(example, tokenize=False) for example in examples
    ]  # Prepare texts for processing
    image_inputs = [process_vision_info(example)[0] for example in examples]  # Process the images to extract inputs

    # Tokenize the texts and process the images
    batch = processor(
        text=texts, images=image_inputs, return_tensors="pt", padding=True
    )  # Encode texts and images into tensors

    # The labels are the input_ids, and we mask the padding tokens in the loss computation
    labels = batch["input_ids"].clone()  # Clone input IDs for labels
    labels[labels == processor.tokenizer.pad_token_id] = -100  # Mask padding tokens in labels

    # Ignore the image token index in the loss computation (model specific)
    if isinstance(processor, Qwen2VLProcessor):  # Check if the processor is Qwen2VLProcessor
        image_tokens = [151652, 151653, 151655]  # Specific image token IDs for Qwen2VLProcessor
    else:
        image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]  # Convert image token to ID

    # Mask image token IDs in the labels
    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100  # Mask image token IDs in labels

    batch["labels"] = labels  # Add labels to the batch

    return batch  # Return the prepared batch

In [18]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn,
    #tokenizer=processor.tokenizer,
)

In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
/venv/main/lib/python3.10/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
30,0.736100,0.220423
60,0.123500,0.099114
90,0.088400,0.068127
120,0.073400,0.052605
150,0.058700,0.046595
180,0.052500,0.044232
210,0.047100,0.041257
240,0.045900,0.040238
270,0.043900,0.039574
300,0.044100,0.039552


/venv/main/lib/python3.10/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
/venv/main/lib/python3.10/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
/venv/main/lib/python3.10/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
/venv/main/lib/python3.10/site-packages/torch/uti

TrainOutput(global_step=319, training_loss=0.1256866062696451, metrics={'train_runtime': 5147.8073, 'train_samples_per_second': 0.991, 'train_steps_per_second': 0.062, 'total_flos': 1.1013663369272525e+17, 'train_loss': 0.1256866062696451})

In [ ]:
trainer.push_to_hub()